In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

In [2]:
def load_data(imu_path, video_path):
    print(imu_path, video_path)
    imu_df = pd.read_csv(imu_path)
    video_df = pd.read_csv(video_path)

    imu_df = imu_df.sort_values("Timestamp")
    video_df = video_df.sort_values("Timestamp")

    return imu_df, video_df

In [3]:
def merge_imu_video(imu_df, video_df):
    merged_df = pd.merge_asof(video_df, imu_df, on="Timestamp")
    return merged_df

In [107]:
participant_data = []
imu_path = './processed/annotated/'
video_path = './processed/metadata/'
for i in range(7,8):  # Loop through 10 participants
    
    imu_df, video_df = load_data(imu_path+os.listdir(imu_path)[i], video_path+os.listdir(video_path)[i])
    merged_df = merge_imu_video(imu_df, video_df)
    
    participant_data.append(merged_df)  # Store each participant’s merged data

./processed/annotated/participant_08.csv ./processed/metadata/participant_08.csv


In [108]:
final_df = pd.concat(participant_data, ignore_index=True)

In [109]:
final_df.head()

,Unnamed: 0,Timestamp,frame_path,QuatI_0,QuatJ_0,QuatK_0,QuatSum_0,QuatI_1,QuatJ_1,QuatK_1,...,QuatSum_3,QuatI_4,QuatJ_4,QuatK_4,QuatSum_4,QuatI_5,QuatJ_5,QuatK_5,QuatSum_5,Activity
0,0,0.000,../../video_data/frames/participant_8\frame_02...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.032,../../video_data/frames/participant_8\frame_02...,-4.329637e-17,-0.707107,0.707107,-4.329637e-17,-0.548061,-0.781216,0.011013,...,0.404997,-4.329637e-17,-0.707107,0.707107,-4.329637e-17,0.174159,-0.764074,0.025792,-0.620641,Unknown
2,2,0.034,../../video_data/frames/participant_8\frame_02...,-4.329637e-17,-0.707107,0.707107,-4.329637e-17,-0.548061,-0.781216,0.011013,...,0.404997,-4.329637e-17,-0.707107,0.707107,-4.329637e-17,0.174159,-0.764074,0.025792,-0.620641,Unknown
3,3,0.062,../../video_data/frames/participant_8\frame_02...,-4.329637e-17,-0.707107,0.707107,-4.329637e-17,-0.548061,-0.781216,0.011013,...,0.404757,-4.329637e-17,-0.707107,0.707107,-4.329637e-17,0.174154,-0.764079,0.024027,-0.620707,Unknown
4,4,0.068,../../video_data/frames/participant_8\frame_02...,-4.329637e-17,-0.707107,0.707107,-4.329637e-17,-0.548061,-0.781216,0.011013,...,0.404757,-4.329637e-17,-0.707107,0.707107,-4.329637e-17,0.174154,-0.764079,0.024027,-0.620707,Unknown


In [110]:
X_imu = final_df[[col for col in final_df.columns if 'Quat' in col]].values
X_imu[:5]

array([[            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan],
       [-4.32963729e-17, -7.07106781e-01,  7.07106781e-01,
        -4.32963729e-17, -5.48061454e-01, -7.81215750e-01,
         1.10131137e-02,  2.98679269e-01, -5.01852824e-01,
        -7.92074992e-01,  1.94309489e-02,  3.46963094e-01,
        -6.83137702e-01, -5.64607428e-01,  2.24763678e-01,
         4.04997064e-01, -4.32963729e-17, -7.07106781e-01,
         7.07106781e-01, -4.32963729e-17,  1.74158917e-01,
        -7.64073504e-01,  2.57922544e-02, -6.20640887e-01],
       [-4.32963729e-17, -7.07106781e-01,  7.07106781e

In [111]:
X_videopaths = final_df["frame_path"].values
X_videopaths[:5]

array(['../../video_data/frames/participant_8\\frame_02_0.0.jpg',
       '../../video_data/frames/participant_8\\frame_02_0.032.jpg',
       '../../video_data/frames/participant_8\\frame_02_0.034.jpg',
       '../../video_data/frames/participant_8\\frame_02_0.062.jpg',
       '../../video_data/frames/participant_8\\frame_02_0.068.jpg'],
      dtype=object)

In [112]:
y_train = final_df["Activity"].values
y_train[:5]

array([nan, 'Unknown', 'Unknown', 'Unknown', 'Unknown'], dtype=object)

## **Extracting image features from a pretrained model**

In [27]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [113]:
feature_extractor = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

C:\Users\ravik\AppData\Local\Temp\ipykernel_8864\1104203163.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  feature_extractor = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


In [ ]:
def extract_image_features(image_paths):
    features = []
    for path in image_paths:
        img = load_img(path, target_size=(224, 224))

        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) #to adding batch dimension
        img_array = img_array/255.0 #normalizing pixel values between 0 and 1

        feature = feature_extractor.predict(img_array)
        features.append(feature.flatten())

    return np.array(features)

image_features = extract_image_features(X_videopaths)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━

In [101]:
image_features[:1]

array([[0.793925  , 0.00349467, 0.13809054, ..., 0.44282025, 0.08942101,
        0.31917128]], dtype=float32)

In [102]:
image_features.shape

(19994, 1280)

In [103]:
final_df_filtered = final_df.copy()
final_df_filtered["Image_Features"] = list(image_features)


In [104]:
print(final_df_filtered.columns)

Index(['Unnamed: 0', 'Timestamp', 'frame_path', 'QuatI_0', 'QuatJ_0',
       'QuatK_0', 'QuatSum_0', 'QuatI_1', 'QuatJ_1', 'QuatK_1', 'QuatSum_1',
       'QuatI_2', 'QuatJ_2', 'QuatK_2', 'QuatSum_2', 'QuatI_3', 'QuatJ_3',
       'QuatK_3', 'QuatSum_3', 'QuatI_4', 'QuatJ_4', 'QuatK_4', 'QuatSum_4',
       'QuatI_5', 'QuatJ_5', 'QuatK_5', 'QuatSum_5', 'Activity',
       'Image_Features'],
      dtype='object')


In [105]:
final_df_filtered = final_df_filtered.drop(columns=["Unnamed: 0"]) 
print(final_df_filtered.columns)

Index(['Timestamp', 'frame_path', 'QuatI_0', 'QuatJ_0', 'QuatK_0', 'QuatSum_0',
       'QuatI_1', 'QuatJ_1', 'QuatK_1', 'QuatSum_1', 'QuatI_2', 'QuatJ_2',
       'QuatK_2', 'QuatSum_2', 'QuatI_3', 'QuatJ_3', 'QuatK_3', 'QuatSum_3',
       'QuatI_4', 'QuatJ_4', 'QuatK_4', 'QuatSum_4', 'QuatI_5', 'QuatJ_5',
       'QuatK_5', 'QuatSum_5', 'Activity', 'Image_Features'],
      dtype='object')


In [106]:
final_df_filtered.to_csv("./processed/multimodal/participant_07.csv")

In [ ]:
df_loaded = pd.read_csv("imu_video_features.csv")

# Convert image features back to NumPy array
df_loaded["Image_Features"] = df_loaded["Image_Features"].apply(lambda x: np.array(eval(x)))